In [1]:
import pandas as pd
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain.agents.agent_types import AgentType
import httpx
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_core.messages import AIMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_community.agent_toolkits.sql.prompt import SQL_FUNCTIONS_SUFFIX
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_openai_functions_agent
import sqlite3

In [2]:
API_KEY = ""

In [3]:
conn = sqlite3.connect("appointments.db")
cursor = conn.cursor()

In [4]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS doctors (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    specialization TEXT NOT NULL,
    contact TEXT NOT NULL
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS slots (
    id INTEGER,
    doctor_id INTEGER NOT NULL,
    datetime TEXT NOT NULL,
    is_available BOOLEAN NOT NULL,
    FOREIGN KEY (doctor_id) REFERENCES doctors (id)
)
""")

In [5]:
cursor.execute("INSERT INTO doctors (name, specialization, contact) VALUES ('Michael Smith', 'Cardiology', '1234567890')")
cursor.execute("INSERT INTO doctors (name, specialization, contact) VALUES ('Ronny Lee', 'Dermatology', '9876543210')")

doctor_ids = {row[0]: row[1] for row in cursor.execute("SELECT name, id FROM doctors").fetchall()}

cursor.execute("INSERT INTO slots (doctor_id, datetime, is_available) VALUES (?, '2024-12-22 10:00 AM', 1)", (doctor_ids['Michael Smith'],))
cursor.execute("INSERT INTO slots (doctor_id, datetime, is_available) VALUES (?, '2024-12-22 11:00 AM', 0)", (doctor_ids['Michael Smith'],))
cursor.execute("INSERT INTO slots (doctor_id, datetime, is_available) VALUES (?, '2024-12-22 1:00 PM', 1)", (doctor_ids['Ronny Lee'],))
cursor.execute("INSERT INTO slots (doctor_id, datetime, is_available) VALUES (?, '2024-12-22 3:00 PM', 1)", (doctor_ids['Ronny Lee'],))

conn.commit()
conn.close()

In [33]:
memory = ConversationBufferMemory(memory_key="history")
llm = ChatOpenAI(openai_api_key=API_KEY, temperature=0, model="gpt-4o", max_tokens=500, http_client = httpx.Client(verify=False))

In [34]:
db = SQLDatabase.from_uri("sqlite:///appointments.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
context = toolkit.get_context()
tools = toolkit.get_tools()

messages = [
            HumanMessagePromptTemplate.from_template("{input}"),
            AIMessage(content=SQL_FUNCTIONS_SUFFIX + "\nUse like operator with lowercase when matching a name\nWhen user is asking to book slots, delete the corresponding row from the table\n"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
            ]
prompt = ChatPromptTemplate.from_messages(messages)
prompt = prompt.partial(**context)
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
            agent=agent,
            tools=toolkit.get_tools(),
            verbose=True,
        )

In [35]:
user_input = 'book 3PM slots for dr lee'

In [36]:
output = agent_executor.invoke({"input": user_input})



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`


doctors, slots
Invoking: `sql_db_schema` with `{'table_names': 'doctors, slots'}`



CREATE TABLE doctors (
	id INTEGER, 
	name TEXT NOT NULL, 
	specialization TEXT NOT NULL, 
	contact TEXT NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from doctors table:
id	name	specialization	contact
1	Michael Smith	Cardiology	1234567890
2	Ronny Lee	Dermatology	9876543210
*/


CREATE TABLE slots (
	id INTEGER, 
	doctor_id INTEGER NOT NULL, 
	datetime TEXT NOT NULL, 
	is_available BOOLEAN NOT NULL, 
	FOREIGN KEY(doctor_id) REFERENCES doctors (id)
)

/*
3 rows from slots table:
id	doctor_id	datetime	is_available
None	1	2024-12-22 10:00 AM	True
None	1	2024-12-22 11:00 AM	False
None	2	2024-12-22 1:00 PM	True
*/
Invoking: `sql_db_query` with `{'query': "SELECT slots.id, slots.datetime FROM slots JOIN doctors ON slots.doctor_id = doctors.id WHERE doctors.name LIKE '%lee%' AND slots.datetime LIKE '%3:00 PM%' AND slots.is_availab

In [37]:
print(output['output'])

The 3:00 PM slot for Dr. Lee on December 22, 2024, has been successfully booked for you.
